<a href="https://colab.research.google.com/github/yingzibu/drug_design_JAK/blob/main/TorchDrug_Molecule_Generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


## Manual Steps

1.   Get your own copy of this file via "File > Save a copy in Drive...",
2.   Set the runtime to **GPU** via "Runtime > Change runtime type..."

### Colab Tutorials

#### Quick Start
1. [Basic Usage and Pipeline](https://colab.research.google.com/drive/1Tbnr1Fog_YjkqU1MOhcVLuxqZ4DC-c8-#forceEdit=true&sandboxMode=true)

#### Drug Discovery Tasks
1. [Property Prediction](https://colab.research.google.com/drive/1sb2w3evdEWm-GYo28RksvzJ74p63xHMn?usp=sharing#forceEdit=true&sandboxMode=true)
2. [Pretrained Molecular Representations](https://colab.research.google.com/drive/10faCIVIfln20f2h1oQk2UrXiAMqZKLoW?usp=sharing#forceEdit=true&sandboxMode=true)
3. [De Novo Molecule Design](https://colab.research.google.com/drive/1JEMiMvSBuqCuzzREYpviNZZRVOYsgivA?usp=sharing#forceEdit=true&sandboxMode=true)
4. [Retrosynthesis](https://colab.research.google.com/drive/1IH1hk7K3MaxAEe5m6CFY7Eyej3RuiEL1?usp=sharing#forceEdit=true&sandboxMode=true)
5. [Knowledge Graph Reasoning](https://colab.research.google.com/drive/1-sjqQZhYrGM0HiMuaqXOiqhDNlJi7g_I?usp=sharing#forceEdit=true&sandboxMode=true)

In [ ]:
import os
import torch
os.environ["TORCH_VERSION"] = torch.__version__

!pip install torch-scatter torch-cluster -f https://pytorch-geometric.com/whl/torch-$TORCH_VERSION.html
!pip install torchdrug

Looking in links: https://pytorch-geometric.com/whl/torch-1.10.0+cu111.html
     |████████████████████████████████| 7.9 MB 4.5 MB/s 
     |████████████████████████████████| 191 kB 4.3 MB/s 
     |████████████████████████████████| 108 kB 32.6 MB/s 
     |████████████████████████████████| 22.3 MB 1.7 MB/s 


# Molecule Generation
Molecular graph generation is a fundamental problem for drug discovery and has been attracting growing attention. The problem is challenging since it requires not only generating chemically valid molecular structures but also optimizing their chemical properties in the meantime.

In this tutorial, we will implement two graph generative models **GCPN** and **GraphAF**. We first pretrain both models on ZINC250k dataset. Starting from the pretrained checkpoint, we finetune both models with reinforcement learning to optimize two properties (i.e., QED and penalized logP score) of generated molecules.

[**Additional reading:** a list of papers related to molecule generation](https://lists.papersapp.com/D0ZUnFtekf_9)

## Prepare the dataset (~8 min)
The `torchdrug.datasets` contains a lot of common machine-learning ready datasets for various drug discovery tasks. It will automatically download the dataset to the specified directory and process it.

We use ZINC250k for this tutorial, and the dataset contains 250,000 drug-like molecules with a maximum atom number of 38. It has 9 atom types and 3 edge types.

For illustration purpose, we can also inherit the ZINC250k and use a small portion of the data (e.g. ZINC10k).

In [ ]:
import torch
from torchdrug import datasets, core, models, tasks, utils
from torch import optim

# The following functions takes 8 mins
dataset = datasets.ZINC250k("~/molecule-datasets/", kekulize=True,
                            node_feature="symbol")

16:11:55   Downloading https://raw.githubusercontent.com/aspuru-guzik-group/chemical_vae/master/models/zinc_properties/250k_rndm_zinc_drugs_clean_3.csv to /root/molecule-datasets/250k_rndm_zinc_drugs_clean_3.csv


Loading /root/molecule-datasets/250k_rndm_zinc_drugs_clean_3.csv:  35%|███▌      | 175050/498911 [00:04<00:10, 29846.41it/s]

## [GCPN](https://arxiv.org/abs/1806.02473)
Graph Convolutional Policy Network (GCPN) is a general graph convolutional network based model for goal-directed molecule generation through reinforcement learning. The [official code](https://github.com/bowenliu16/rl_graph_generation) of the model has more than one thousand lines.
Here, we will show how to construct the GCPN model, pretrain and finetune it in a few lines.

### Construct the GCPN model (~30s)
The model consists of two parts, a graph representation model and a graph generative module.

(1) We define a Relational Graph Convolutional Networks (RGCN) as our representation model.

$\mathbf{x}^{\prime}_i = \mathbf{\theta}_{\textrm{root}} \cdot \mathbf{x}_i + \sum_{r \in \mathcal{R}} \sum_{j \in \mathcal{N}_r(i)} \frac{1}{|\mathcal{N}_r(i)|} \mathbf{\theta}_r \cdot \mathbf{x}_j$, where $\mathcal{R}$ denotes the set of relations, i.e. edge types

(2) We use the module GCPNGeneration as the training task for GCPN.

In [ ]:
dataset.transform = None
# (1)
model = models.RGCN(input_dim=dataset.node_feature_dim,
                    num_relation=dataset.num_bond_type,
                    hidden_dims=[256, 256, 256, 256], batch_norm=False)
# (2)
task = tasks.GCPNGeneration(model, dataset.atom_types, max_edge_unroll=12,
                            max_node=38, criterion="nll")

### Training (~ 10 min / epoch on Colab)
Now we can train our model. We setup an optimizer for our model, and put everything together into an Engine instance. Here we only train the model for 1 epoch, and then save the pretrained model into a directory.

Like other deep learning libraries, these codes are always reusable, and you can focus on developing interesting models.


In [ ]:
optimizer = optim.Adam(task.parameters(), lr = 1e-3)
solver = core.Engine(task, dataset, None, None, optimizer,
                     gpus=(0,), batch_size=128, log_interval=10)
solver.train(num_epoch=1)
solver.save("gcpn_zinc250k_1epoch.pkl")

### Download the pretrained checkpoint and load the checkpoint.
We provide a pretrained model (5 epoch on ZINC250k) in a [google drive folder](https://drive.google.com/file/d/11iHZGSXY0Bw7K5DHlG06kwF2GDT-8-iq/view?usp=sharing). We can easily get the checkpoint via `gdown` command given the `file_id`. We then load the parameters from the checkpoint in one line.

In [ ]:
!gdown --id 11iHZGSXY0Bw7K5DHlG06kwF2GDT-8-iq
solver.load('gcpn_zinc250k_5epoch.pkl')

Let’s generate some small molecules from the pretrained GCPN model.

In [ ]:
%matplotlib inline
results = task.generate(num_sample=32, max_resample=5)
results.visualize(num_row=4, num_col=None, save_file=None, titles=None)

### Finetuning
Now we introduce how to fine-tune the graph generative model with reinforcement learning to optimize the properties of generated molecules. We implemented the Proximal Policy Optimization (PPO) algorithm for the GCPN model.

Lets first re-construct the GCPN model and load the pretrained model. Note that we reset the `task` and the `criterion` for the gcpn model

In [ ]:
# define the task and load the pretrained model
dataset.transform = None
model = models.RGCN(input_dim=dataset.node_feature_dim,
                    num_relation=dataset.num_bond_type,
                    hidden_dims=[256, 256, 256, 256], batch_norm=False)
task = tasks.GCPNGeneration(model, dataset.atom_types, max_edge_unroll=12,
                            max_node=38, task="plogp", criterion="ppo",
                            reward_temperature=1,
                            agent_update_interval=3, gamma=0.9)
optimizer = optim.Adam(task.parameters(), lr = 1e-5)
solver = core.Engine(task, dataset, None, None, optimizer,
                     gpus=(0,), batch_size=16, log_interval=10)
solver.load('gcpn_zinc250k_5epoch.pkl', load_optimizer=False)

Now lets finetune the model. The model will generated molecules with desired property during the finetuning.

In [ ]:
# RL finetuning
solver.train(num_epoch=1)

## [GraphAF](https://arxiv.org/abs/2001.09382)

GraphAF is a flow-based autoregressive model for graph generation. Nodes and edges are added to the existing graphs autoregressively.

![graphaf.png](https://raw.githubusercontent.com/DeepGraphLearning/torchdrug/master/asset/model/graphaf.png)


### Construct GraphAF model (~30s)
The model consists of two parts, a graph representation model and a graph generative module. We define a Relational Graph Convolutional Networks (RGCN) as our representation model. We use the module AutoregressiveGeneration as the training task for GraphAF. The task consists of a node flow model and an edge flow model, which define invertible mapping between node / edge types and noise distributions.

In [ ]:
from torchdrug.layers import distribution
dataset.transform = None
model = models.RGCN(input_dim=dataset.num_atom_type,
                    num_relation=dataset.num_bond_type,
                    hidden_dims=[256, 256, 256], batch_norm=True)

num_atom_type = dataset.num_atom_type
# add one class for non-edge
num_bond_type = dataset.num_bond_type + 1

node_prior = distribution.IndependentGaussian(torch.zeros(num_atom_type),
                                              torch.ones(num_atom_type))
edge_prior = distribution.IndependentGaussian(torch.zeros(num_bond_type),
                                              torch.ones(num_bond_type))
node_flow = models.GraphAF(model, node_prior, num_layer=12)
edge_flow = models.GraphAF(model, edge_prior, use_edge=True, num_layer=12)

task = tasks.AutoregressiveGeneration(node_flow, edge_flow,
                                      max_node=38, max_edge_unroll=12,
                                      criterion="nll")

### Pre-Training (10-20 mins / epoch on Colab)
Now we can train our model. We setup an optimizer for our model, and put everything together into an Engine instance. Here we only train the model for 1 epoch, and then save the pretrained model into a directory.

In [ ]:
optimizer = optim.Adam(task.parameters(), lr = 1e-3)
solver = core.Engine(task, dataset, None, None, optimizer,
                     gpus=(0,), batch_size=128, log_interval=10)

solver.train(num_epoch=1)
solver.save("graphaf_zinc250k_1epoch.pkl")

### Download the pretrained checkpoint and load the checkpoint. (~30s)
We provide a pretrained model (5 epoch on ZINC250k) in a [google drive folder](https://drive.google.com/file/d/1iIlzFGrhUrB2yt7ettOkcHcmKPevT1_Y/view?usp=sharing). We can easily get the checkpoint via `gdown` command given the `file_id`. We then load the parameters from the checkpoint.

In [ ]:
!gdown --id 1iIlzFGrhUrB2yt7ettOkcHcmKPevT1_Y
solver.load('graphaf_zinc250k_5epoch.pkl')

### Generation
Let’s generate some small molecules using the pretrained GraphAF model.

In [ ]:
%matplotlib inline
results = task.generate(num_sample=16, max_resample=10)
results.visualize(num_row=4, num_col=None, save_file=None, titles=None)

### Fine-Tuning
Now we introduce how to fine-tune the graph generative model with reinforcement learning to optimize the properties of generated molecules. We implemented the Proximal Policy Optimization (PPO) algorithm for the GraphAF model.

Lets first re-construct the GraphAF model and load the pretrained model. Note that we reset the `task` and the `criterion` for the model.

In [ ]:
dataset.transform = None
model = models.RGCN(input_dim=dataset.num_atom_type,
                    num_relation=dataset.num_bond_type,
                    hidden_dims=[256, 256, 256], batch_norm=True)
node_prior = distribution.IndependentGaussian(torch.zeros(num_atom_type),
                                              torch.ones(num_atom_type))
edge_prior = distribution.IndependentGaussian(torch.zeros(num_bond_type),
                                              torch.ones(num_bond_type))
node_flow = models.GraphAF(model, node_prior, num_layer=12)
edge_flow = models.GraphAF(model, edge_prior, use_edge=True, num_layer=12)

task = tasks.AutoregressiveGeneration(node_flow, edge_flow,
                                      max_node=38, max_edge_unroll=12,
                                      task="plogp", criterion="ppo",
                                      reward_temperature=20, baseline_momentum=0.9,
                                      agent_update_interval=5, gamma=0.9)
optimizer = optim.Adam(task.parameters(), lr=1e-5)
solver = core.Engine(task, dataset, None, None, optimizer,
                     gpus=(0,), batch_size=64, log_interval=10)
solver.load('graphaf_zinc250k_5epoch.pkl', load_optimizer=False)


Now lets finetune the model. The model will generated molecules with desired property during the finetuning.

In [ ]:
# RL finetuning
solver.train(num_epoch=10)